In [1]:
!pip install topicnet
!pip install bigartm
!pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.7/104.7 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.2 MB/s eta 0:00:0000:01:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.9/123.9 kB 10.4 MB/s eta 0:00:00
  Created wheel for topicnet: filename=topicnet-0.8.0-py3-none-any.whl size=134578 sha256=daa319c286ff12b107adebadfc669d7e84c100a4d3f4bafb69fc5b7e429184ee
  Stored in directory: /root/.cache/pip/wheels/c9/b9/4e/80a7bf4d79ad575dfe37c8a11fba7edd5992b8ec0c59784139
Successfully built topicnet
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 60.0 MB/s eta 0:00:0000:0100:01


In [2]:
import artm

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

sns.set(style="darkgrid", palette="Set2")

from topicnet.dataset_manager import api

# topicnet imports
from topicnet.cooking_machine.cubes import (
    CubeCreator,
    GreedyStrategy,
    PerplexityStrategy,
    RegularizationControllerCube,
    RegularizersModifierCube,
)
from topicnet.cooking_machine.dataset import Dataset
from topicnet.cooking_machine.experiment import Experiment
from topicnet.cooking_machine.models import BaseScore
from topicnet.cooking_machine.models.topic_model import TopicModel
from topicnet.cooking_machine.model_constructor import add_standard_scores
from topicnet.cooking_machine.model_constructor import init_simple_default_model
from topicnet.cooking_machine.pretty_output import make_notebook_pretty
from topicnet.viewers.top_documents_viewer import TopDocumentsViewer
from topicnet.viewers.top_tokens_viewer import TopTokensViewer

from IPython.display import display, display_html

# for preprocessing
import nltk

from nltk.collocations import (
    BigramAssocMeasures,
    BigramCollocationFinder,
)
from nltk.corpus import (
    stopwords,
    wordnet,
)

nltk.download("wordnet")
nltk.download("omw-1.4")
nltk.download("averaged_perceptron_tagger_ru")
nltk.download("stopwords")

from nltk.stem import WordNetLemmatizer

from pymorphy2 import MorphAnalyzer

from collections import Counter

from tqdm import tqdm

from topicnet.cooking_machine.model_constructor import (
    create_default_topics,
    count_vocab_size,
    init_model,
)
from topicnet.cooking_machine.rel_toolbox_lite import (
    count_vocab_size,
    modality_weight_rel2abs,
)

from typing import Dict, List

from time import process_time

/opt/conda/lib/python3.7/site-packages/nltk/decorators.py:70: DeprecationWarning:

`formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly



[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Предобработка данных

Создадим таблицу с данными по новостям с сайта lenta.ru.

In [3]:
df_lenta = pd.read_csv(
    "/kaggle/input/corpus-of-russian-news-articles-from-lenta/lenta-ru-news.csv"
)
df_lenta

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning:

Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.



,url,title,text,topic,tags,date
0,https://lenta.ru/news/1914/09/16/hungarnn/,1914. Русские войска вступили в пределы Венгрии,Бои у Сопоцкина и Друскеник закончились отступ...,Библиотека,Первая мировая,1914/09/16
1,https://lenta.ru/news/1914/09/16/lermontov/,1914. Празднование столетия М.Ю. Лермонтова от...,"Министерство народного просвещения, в виду про...",Библиотека,Первая мировая,1914/09/16
2,https://lenta.ru/news/1914/09/17/nesteroff/,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в...",Библиотека,Первая мировая,1914/09/17
3,https://lenta.ru/news/1914/09/17/bulldogn/,1914. Бульдог-гонец под Льежем,Фотограф-корреспондент Daily Mirror рассказыва...,Библиотека,Первая мировая,1914/09/17
4,https://lenta.ru/news/1914/09/18/zver/,1914. Под Люблином пойман швабский зверь,"Лица, приехавшие в Варшаву из Люблина, передаю...",Библиотека,Первая мировая,1914/09/18
...,...,...,...,...,...,...
800970,https://lenta.ru/news/2019/12/14/shnur/,Шнуров раскритиковал Гагарину на «Голосе»,Певец Сергей Шнуров раскритиковал свою коллегу...,NaN,ТВ и радио,2019/12/14
800971,https://lenta.ru/news/2019/12/14/dolg/,В России предложили изменить правила взыскания...,Министерство юстиции России предложило изменит...,NaN,Все,2019/12/14
800972,https://lenta.ru/news/2019/12/14/dark_euro/,В России назвали «черную дату» для Европы,Испытание США ранее запрещенной Договором о ли...,NaN,Политика,2019/12/14
800973,https://lenta.ru/news/2019/12/14/meteo/,Россиянам пообещали аномально теплую погоду,В ближайшие дни в европейской части России пог...,NaN,Общество,2019/12/14


Приводим столбец с данными в необходимый формат.

In [4]:
df_lenta.date = pd.to_datetime(df_lenta.date)
df_lenta.date

0        1914-09-16
1        1914-09-16
2        1914-09-17
3        1914-09-17
4        1914-09-18
            ...    
800970   2019-12-14
800971   2019-12-14
800972   2019-12-14
800973   2019-12-14
800974   2019-12-14
Name: date, Length: 800975, dtype: datetime64[ns]

Для исследования рассмотрим все новости, опубликованные в 2008 году.

In [18]:
df_lenta_short = df_lenta[
    (df_lenta["date"].dt.year == 2008)
    & (df_lenta["date"].dt.month >= 5)
    & (df_lenta["date"].dt.month <= 8)
]
df_lenta_short

,url,title,text,topic,tags,date
227572,https://lenta.ru/news/2008/05/01/bus/,В Египте загорелся автобус с туристами из России,По меньшей мере семь иностранных туристов поги...,Мир,Все,2008-05-01
227573,https://lenta.ru/news/2008/05/01/sudden/,"Рядовой-срочник получил ""внезапное"" ранение на...",В ночь на четверг в Назрановском районе Ингуше...,Россия,Все,2008-05-01
227574,https://lenta.ru/news/2008/05/01/bars/,Бежавшая из тюрьмы американка арестована спуст...,"Женщина, в течение 32 лет скрывавшаяся от поли...",Россия,Все,2008-05-01
227575,https://lenta.ru/news/2008/05/01/gisele/,Жизель Бюндхен не уступила первое место в рейт...,Бразильская топ-модель Жизель Бюндхен (Gisele ...,Из жизни,Все,2008-05-01
227576,https://lenta.ru/news/2008/05/01/raf/,За год британские ВВС более 20 раз вылетали на...,Самолеты ВВС Великобритании за последний год н...,Мир,Все,2008-05-01
...,...,...,...,...,...,...
244016,https://lenta.ru/news/2008/08/31/tennis/,Динара Сафина вышла в 1/8 финала Открытого чем...,Россиянка Динара Сафина в третьем раунде Откры...,Спорт,Все,2008-08-31
244017,https://lenta.ru/news/2008/08/31/quake/,"На Байкале произошло землетрясение в 6,4 балла",В районе озера Байкал в ночь на воскресенье пр...,Мир,Все,2008-08-31
244018,https://lenta.ru/news/2008/08/31/two/,В Кабардино-Балкарии застрелили двух милиционеров,"В Кабардино-Балкарии убиты два милиционера, ещ...",Россия,Все,2008-08-31
244019,https://lenta.ru/news/2008/08/31/pigeon/,Британцы собирались использовать голубей для д...,Британская разведывательная служба MI5 планиро...,Мир,Все,2008-08-31


Посмотрим, какие макротемы содержатся в датасете. В ходе исследования будут выделены более конкретные темы.

In [19]:
df_lenta_short.groupby("topic").count().index

Index(['Бывший СССР', 'Дом', 'Из жизни', 'Интернет и СМИ', 'Культура', 'Мир',
       'Наука и техника', 'Россия', 'Силовые структуры', 'Спорт', 'Экономика'],
      dtype='object', name='topic')

Далее оставим в датасете необходимые данные.

In [20]:
df_lenta_raw = pd.DataFrame(df_lenta_short[["title", "text", "topic", "tags"]]).rename(
    columns={"text": "raw_text"}
)
df_lenta_raw

,title,raw_text,topic,tags
227572,В Египте загорелся автобус с туристами из России,По меньшей мере семь иностранных туристов поги...,Мир,Все
227573,"Рядовой-срочник получил ""внезапное"" ранение на...",В ночь на четверг в Назрановском районе Ингуше...,Россия,Все
227574,Бежавшая из тюрьмы американка арестована спуст...,"Женщина, в течение 32 лет скрывавшаяся от поли...",Россия,Все
227575,Жизель Бюндхен не уступила первое место в рейт...,Бразильская топ-модель Жизель Бюндхен (Gisele ...,Из жизни,Все
227576,За год британские ВВС более 20 раз вылетали на...,Самолеты ВВС Великобритании за последний год н...,Мир,Все
...,...,...,...,...
244016,Динара Сафина вышла в 1/8 финала Открытого чем...,Россиянка Динара Сафина в третьем раунде Откры...,Спорт,Все
244017,"На Байкале произошло землетрясение в 6,4 балла",В районе озера Байкал в ночь на воскресенье пр...,Мир,Все
244018,В Кабардино-Балкарии застрелили двух милиционеров,"В Кабардино-Балкарии убиты два милиционера, ещ...",Россия,Все
244019,Британцы собирались использовать голубей для д...,Британская разведывательная служба MI5 планиро...,Мир,Все


Переводим в формат, необходимый для работы с TopicNet. Выделим две модальности:

- `lemmatized` отвечает за слова в тексте и заголовке в начальной форме
- `bigram` отвечает за биграммы в тексте

In [8]:
def vowpalize_sequence(sequence):
    """
    Переводит последовательность в формат из Vowpal Wabbit.

    Аргумент:
    sequence - последовательность

    Возвращает:
    res - результат преобразования
    """
    word_freq = Counter(sequence)
    del word_freq[""]

    res = ""
    for word in word_freq:
        res += word + ":" + str(word_freq[word]) + " "

    return res


def process_data(dataframe, vocab_size=16000):
    """
    Отвечает за предобработку набора данных для
    дальнейшего использования в TopicNet.

    Аргумент:
    dataframe - данные, содержащие колонку raw_text
    vocab_size - размер словаря биграмм

    Результат:
    dataframe - предобработанные данные
    """
    tokenized_text = []  # инициализируем список

    # разбиваем на токены
    for _, data in tqdm(dataframe.iterrows()):
        tokens = [
            token
            for token in nltk.wordpunct_tokenize(data.title.lower())
            if len(token) > 1
        ]
        tokens.extend(
            [
                token
                for token in nltk.wordpunct_tokenize(data.raw_text.lower())
                if len(token) > 1
            ]
        )
        tokenized_text.append(tokens)

    # запоминаем токенизацию
    dataframe["tokenized"] = tokenized_text

    # список стопслов
    stop = set(stopwords.words("russian"))

    # применяем лемматизацию, используем PyMorphy2
    lemmatized_text = []
    morph = MorphAnalyzer()

    # переводим в нормальную форму
    for text in tqdm(dataframe["tokenized"].values):
        lemmatized = [morph.parse(word)[0].normal_form for word in text]
        lemmatized = [
            word for word in lemmatized if word not in stop and word.isalpha()
        ]
        lemmatized_text.append(lemmatized)

    dataframe["lemmatized"] = lemmatized_text

    # выбираем лучшие биграммы
    bigram_measures = BigramAssocMeasures()
    finder = BigramCollocationFinder.from_documents(dataframe["lemmatized"])
    finder.apply_freq_filter(5)
    set_dict = set(finder.nbest(bigram_measures.pmi, vocab_size))
    documents = dataframe["lemmatized"]
    bigrams = []

    # добавляем биграммы
    for doc in tqdm(documents):
        entry = [
            "_".join([word_first, word_second])
            for word_first, word_second in zip(doc[:-1], doc[1:])
            if (word_first, word_second) in set_dict
        ]
        bigrams.append(entry)

    dataframe["bigram"] = bigrams

    # добавялем текст в формате из Vowpal Wabbit
    vw_text = []

    for index, data in tqdm(dataframe.iterrows()):
        vw_string = ""
        doc_id = str(index)
        lemmatized = "@lemmatized " + vowpalize_sequence(data.lemmatized)
        bigram = "@bigram " + vowpalize_sequence(data.bigram)
        vw_string = " |".join([doc_id, lemmatized, bigram])
        vw_text.append(vw_string)

    dataframe["vw_text"] = vw_text

    return dataframe

Ограничим размер словаря в $10 \ 000$ позиций.

In [21]:
df_lenta_vw = process_data(df_lenta_raw, 10000)
df_lenta_vw

16449it [00:03, 4126.79it/s]
/opt/conda/lib/python3.7/site-packages/pymorphy2/units/base.py:70: DeprecationWarning:

inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()

100%|██████████| 16449/16449 [00:00<00:00, 24945.19it/s]
16449it [00:03, 4492.36it/s]


,title,raw_text,topic,tags,tokenized,lemmatized,bigram,vw_text
227572,В Египте загорелся автобус с туристами из России,По меньшей мере семь иностранных туристов поги...,Мир,Все,"[египте, загорелся, автобус, туристами, из, ро...","[египет, загореться, автобус, турист, россия, ...","[малый_мера, травма_ожог, associated_press, пе...",227572 |@lemmatized египет:3 загореться:2 авто...
227573,"Рядовой-срочник получил ""внезапное"" ранение на...",В ночь на четверг в Назрановском районе Ингуше...,Россия,Все,"[рядовой, срочник, получил, внезапное, ранение...","[рядовой, срочник, получить, внезапный, ранени...","[назрановский_район, северный_кавказ, пост_дпс...",227573 |@lemmatized рядовой:3 срочник:3 получи...
227574,Бежавшая из тюрьмы американка арестована спуст...,"Женщина, в течение 32 лет скрывавшаяся от поли...",Россия,Все,"[бежавшая, из, тюрьмы, американка, арестована,...","[бежать, тюрьма, американка, арестовать, спуст...","[сан_диего, досрочный_освобождение, выйти_заму...",227574 |@lemmatized бежать:1 тюрьма:4 американ...
227575,Жизель Бюндхен не уступила первое место в рейт...,Бразильская топ-модель Жизель Бюндхен (Gisele ...,Из жизни,Все,"[жизель, бюндхен, не, уступила, первое, место,...","[жизель, бюндхен, уступить, первый, место, рей...","[строчка_рейтинг, самый_высокооплачиваемый, жу...",227575 |@lemmatized жизель:2 бюндхен:4 уступит...
227576,За год британские ВВС более 20 раз вылетали на...,Самолеты ВВС Великобритании за последний год н...,Мир,Все,"[за, год, британские, ввс, более, 20, раз, выл...","[год, британский, ввс, вылетать, навстречу, бо...","[bbc_news, вооружённый_сила, воздушный_простра...",227576 |@lemmatized год:5 британский:3 ввс:4 в...
...,...,...,...,...,...,...,...,...
244016,Динара Сафина вышла в 1/8 финала Открытого чем...,Россиянка Динара Сафина в третьем раунде Откры...,Спорт,Все,"[динара, сафина, вышла, финала, открытого, чем...","[динар, сафин, выйти, финал, открытый, чемпион...","[динар_сафин, россиянка_динар, динар_сафин, те...",244016 |@lemmatized динар:2 сафин:3 выйти:1 фи...
244017,"На Байкале произошло землетрясение в 6,4 балла",В районе озера Байкал в ночь на воскресенье пр...,Мир,Все,"[на, байкале, произошло, землетрясение, балла,...","[байкал, произойти, землетрясение, балл, район...","[озеро_байкал, эпицентр_землетрясение, населит...",244017 |@lemmatized байкал:3 произойти:2 земле...
244018,В Кабардино-Балкарии застрелили двух милиционеров,"В Кабардино-Балкарии убиты два милиционера, ещ...",Россия,Все,"[кабардино, балкарии, застрелили, двух, милици...","[кабардино, балкария, застрелить, милиционер, ...","[кабардино_балкария, кабардино_балкария, право...",244018 |@lemmatized кабардино:2 балкария:2 зас...
244019,Британцы собирались использовать голубей для д...,Британская разведывательная служба MI5 планиро...,Мир,Все,"[британцы, собирались, использовать, голубей, ...","[британец, собираться, использовать, голубеть,...","[архивный_документ, press_association, bbc_new...",244019 |@lemmatized британец:4 собираться:2 ис...


Экспортируем получившийся после предобработки датасет.

In [22]:
df_lenta_vw.reset_index().rename(columns={"index": "id"}).to_csv(
    "lenta_2008_may_aug_preprocessed.csv"
)